In [2]:
import pandas as pd
import numpy as np


# 读取 Excel 文件
df = pd.read_excel('merged_data.xlsx')


# 保留所需字段和 2023-12 至 2024-11 的所有字段
columns_to_keep = ['id', 'platform', 'repo_name', 'field'] + [col for col in df.columns if '2023-12' <= col <= '2024-11']
df_filtered = df[columns_to_keep]


# 仅保留 field 为 Database、Artificial Intelligence、Cloud Native 的数据
fields_to_keep = ['Database', 'Artificial Intelligence', 'Cloud Native']
df_filtered = df_filtered[df_filtered['field'].isin(fields_to_keep)]


# 重置索引，避免 'field' 成为索引
df_filtered = df_filtered.reset_index(drop=True)


# 处理缺失数据
def handle_missing_data(group):
    # 将 0 视为缺失值，将其替换为 NaN
    group.loc[:, '2023-12':'2024-11'] = group.loc[:, '2023-12':'2024-11'].replace(0, np.nan)
    # 计算 2023-12 至 2024-11 期间缺失数据的个数
    missing_count = group.loc[:, '2023-12':'2024-11'].isna().sum(axis=1)
    # 若缺失数据个数 <= 3，则插值处理，否则删除该条数据
    group = group[missing_count <= 3]
    # 插值处理：线性插值
    group.loc[:, '2023-12':'2024-11'] = group.loc[:, '2023-12':'2024-11'].interpolate(method='linear', limit_direction='both')
    return group


# 使用 groupby 时，避免 field 作为索引
df_filtered = df_filtered.groupby('field', group_keys=False).apply(handle_missing_data)


# 计算每个分组的平均值，保留两位小数
def mean_2023_2024(group):
    return group.loc[:, '2023-12':'2024-11'].mean().round(2)


df_avg = df_filtered.groupby('field', group_keys=False).apply(mean_2023_2024)


# 计算 2023-12 至 2024-11 所有列的总活跃度
df_filtered['total_activity'] = df_filtered.loc[:, '2023-12':'2024-11'].sum(axis=1)


# 根据总活跃度排序，选择前 5 名
df_top_5 = df_filtered.groupby('field', group_keys=False).apply(lambda x: x.nlargest(5, 'total_activity'))


# 输出结果
print("分组平均值：")
print(df_avg)
print("\n每组 top 5：")
print(df_top_5)

分组平均值：
                         2023-12  2024-01  2024-02  2024-03  2024-04  2024-05  \
field                                                                           
Artificial Intelligence   116.09   127.05   113.56   131.17   121.87   115.60   
Cloud Native              103.87   125.48   115.56   121.59   122.75   122.04   
Database                   96.02   107.09    98.18   110.60   106.30   105.50   

                         2024-06  2024-07  2024-08  2024-09  2024-10  2024-11  
field                                                                          
Artificial Intelligence   109.68   118.48   113.29   102.13   103.05    94.27  
Cloud Native              115.46   121.50   117.08   113.94   119.41   107.42  
Database                  100.08   107.45   104.74   103.41   104.10    96.84  

每组 top 5：
           id platform                 repo_name                    field  \
1    65600975   github           pytorch/pytorch  Artificial Intelligence   
16  552661142   github

In [18]:
df_avg

,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11
field,,,,,,,,,,,,
Artificial Intelligence,116.09,127.05,113.56,131.17,121.87,115.60,109.68,118.48,113.29,102.13,103.05,94.27
Cloud Native,103.87,125.48,115.56,121.59,122.75,122.04,115.46,121.50,117.08,113.94,119.41,107.42
Database,96.02,107.09,98.18,110.60,106.30,105.50,100.08,107.45,104.74,103.41,104.10,96.84


In [19]:
df_top_5["2023-12":"2024-11"]

,id,platform,repo_name,field,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,total_activity
1,65600975,github,pytorch/pytorch,Artificial Intelligence,2020.71,2257.89,2429.07,2453.79,2536.57,2585.83,2557.17,2648.74,2845.89,2445.42,2629.82,2486.86,29897.76
16,552661142,github,langchain-ai/langchain,Artificial Intelligence,1766.43,1922.47,1792.93,1906.65,1527.27,1472.22,1517.86,1451.41,1117.90,1080.45,1021.36,856.23,17433.18
3,599547518,github,vllm-project/vllm,Artificial Intelligence,893.04,883.06,909.45,1226.46,1362.67,1243.66,1443.86,1861.39,1973.03,1724.16,1772.42,1600.35,16893.55
7,155220641,github,huggingface/transformers,Artificial Intelligence,1056.89,1225.94,1148.23,1294.57,1212.06,1155.16,1094.84,1297.69,1387.58,1239.29,1382.78,1038.52,14533.55
18,612354784,github,ggerganov/llama.cpp,Artificial Intelligence,914.11,929.78,955.78,1000.29,1151.05,1158.02,910.40,994.85,794.01,706.53,597.75,669.85,10782.42
0,75821432,github,llvm/llvm-project,Cloud Native,3712.05,4733.47,4311.16,4805.10,4789.38,4863.77,4635.97,5168.52,5135.21,4841.52,5072.69,4766.16,56835.00
2,15111821,github,grafana/grafana,Cloud Native,1200.89,1812.85,1839.79,1654.97,1588.21,1533.28,1475.13,1605.69,1537.50,1447.98,1611.32,1534.07,18841.68
6,20580498,github,kubernetes/kubernetes,Cloud Native,963.48,1301.36,1356.98,1300.72,1319.84,1381.97,1363.52,1550.42,1299.59,1480.92,1620.57,1302.28,16241.65
4,60246359,github,ClickHouse/ClickHouse,Cloud Native,925.01,1064.36,1022.43,1185.27,1109.42,1210.63,1081.46,1297.51,1261.23,1012.56,1083.74,1032.56,13286.18
17,11125589,github,keycloak/keycloak,Cloud Native,636.87,840.16,794.50,855.32,841.89,802.49,853.73,834.91,725.78,775.24,984.08,788.08,9733.05
